## Load Dependencies and Paths

In [1]:
!pip install gymnasium
import gymnasium as gym
import sys
sys.modules["gym"] = gym

!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib@feat/gymnasium-support
!pip install stable-baselines3[extra]
!pip install yolov5

  Cloning https://github.com/Stable-Baselines-Team/stable-baselines3-contrib (to revision feat/gymnasium-support) to c:\users\phili\appdata\local\temp\pip-req-build-4vz5idt2
  Resolved https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to commit d93ca7f1a734436cee59d642fa7d3eb8b4ed0253
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 178.4/178.4 kB 2.2 MB/s eta 0:00:00
  Created wheel for sb3-contrib: filename=sb3_contrib-2.0.0a4-py3-none-any.whl size=80746 sha256=876cc87a1c435814873dd04bd8e92115e09ae231e3a34a2ffc40c0c202ad7ad3
  Stored in directory: C:\Users\phili\AppData\Local\Temp\pip-ephem-wheel-cache-v8l6herl\wheels\33\dc\48\f1ae1ed19b0e847f127ea

  Running command git clone --filter=blob:none --quiet https://github.com/Stable-Baselines-Team/stable-baselines3-contrib 'C:\Users\phili\AppData\Local\Temp\pip-req-build-4vz5idt2'
  Running command git checkout -b feat/gymnasium-support --track origin/feat/gymnasium-support
  Branch 'feat/gymnasium-support' set up to track remote branch 'feat/gymnasium-support' from 'origin'.
  Switched to a new branch 'feat/gymnasium-support'


  Using cached ale_py-0.8.1-cp310-cp310-win_amd64.whl (952 kB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.7.4
    Uninstalling ale-py-0.7.4:
      Successfully uninstalled ale-py-0.7.4
     -------------------------------------- 953.3/953.3 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 88.3/88.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 135.8/135.8 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 57.4/57.4 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 104.0/104.0 kB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 11.1/11.1 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 79.8/79.8 kB 4.6 MB/s eta 0:00:00
     --------------------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'D:\\Miniconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Box
import numpy as np
import random
import os
from PIL import Image
from PIL import ImageEnhance
import torch
import sys
sys.modules["gym"] = gym
from stable_baselines3 import SAC, PPO, TD3
import pandas as pd
import gc

In [6]:
# Path to images
train_image_path = "../../GTSDB/images/distorted_hsv"
original_image_path = "../../GTSDB/images"
label_path = "../../GTSDB/labels"

## Create Environment

In [7]:
class Object:
  pass

class DistortionEnv(Env):

    image_width = 320
    image_height = 320


    # Resizes the image to specified dimensions and converts to 3 color channels
    @staticmethod
    def _load_and_convert_image(image_path):
        image = Image.open(image_path)
        image = image.resize((DistortionEnv.image_width, DistortionEnv.image_height))
        if image.mode == "RGBA":
            image = image.convert("RGB")
        return np.asarray(image)

    @staticmethod
    def _calculate_mse(image1, image2):

        # Resize the images if necessary (to ensure they have the same dimensions)
        image1 = image1.resize((DistortionEnv.image_width, DistortionEnv.image_height))
        image2 = image2.resize((DistortionEnv.image_width, DistortionEnv.image_height))

        # Convert the images to grayscale
        image1 = image1.convert("L")
        image2 = image2.convert("L")

        # Convert the images to numpy arrays
        arr1 = np.array(image1)
        arr2 = np.array(image2)

        # Calculate the MSE
        mse = np.mean((arr1 - arr2) ** 2)
        return mse

    # Convert YOLOv5 format labeled bounding boxes to coordinates
    @staticmethod
    def _convert_yolo_to_coordinates(yolo_box, image_width, image_height):
        net_class, center_x, center_y, width, height = yolo_box

        xmin = (center_x - width/2) * image_width
        ymin = (center_y - height/2) * image_height
        xmax = (center_x + width/2) * image_width
        ymax = (center_y + height/2) * image_height
        return net_class, xmin, ymin, xmax, ymax

    def _get_gt_coords_from_yolo_file(self, file_path):
        yolo_boxes = []
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    net_class, center_x, center_y, width, height = line.split()

                    center_x = float(center_x)
                    center_y = float(center_y)
                    width = float(width)
                    height = float(height)

                    entry = (net_class, center_x, center_y, width, height)
                    yolo_boxes.append(entry)

        # Convert Yolo bounding boxes to coordinates
        gt_coords = []
        for yolo_box in yolo_boxes:
            box_coords = self._convert_yolo_to_coordinates(yolo_box, DistortionEnv.image_width, DistortionEnv.image_height)
            gt_coords.append(box_coords)
        return gt_coords

    @staticmethod
    def _calculate_iou(box1, box2):
        class1, xmin1, ymin1, xmax1, ymax1 = box1
        class2, xmin2, ymin2, xmax2, ymax2 = box2

        # Calculate intersection area
        xmin_inter = max(xmin1, xmin2)
        ymin_inter = max(ymin1, ymin2)
        xmax_inter = min(xmax1, xmax2)
        ymax_inter = min(ymax1, ymax2)

        width_inter = max(0, xmax_inter - xmin_inter)
        height_inter = max(0, ymax_inter - ymin_inter)
        area_inter = width_inter * height_inter

        # Calculate union area
        area_box1 = (xmax1 - xmin1) * (ymax1 - ymin1)
        area_box2 = (xmax2 - xmin2) * (ymax2 - ymin2)
        area_union = area_box1 + area_box2 - area_inter

        # Calculate IoU
        iou = area_inter / area_union if area_union > 0 else 0
        return iou


    def _calculate_iou_and_accuracy(self, gt_coords, predictions):
        result = Object()
        result.iou = 0.0
        result.correct_class_ratio = 0.0

        # Iterate over prediction rows, calculate IoU with each ground truth box, keep highest IoU
        iou_values = []
        correct_class_cnt = 0
        for index, row in predictions.iterrows():
            predicted_box = (row['class'], row['xmin'], row['ymin'], row['xmax'], row['ymax'])

            # Calculate IoU with each ground truth box
            iou_per_gt = []
            for gt_coord in gt_coords:
                iou = self._calculate_iou(gt_coord, predicted_box)

                # Check for correct classification if iou bigger than 0.5
                if(iou >= 0.5):
                  if(str(predicted_box[0]) == str(gt_coord[0])):
                    iou_per_gt.append(iou)

            # Store the maximum IoU value for the predicted box
            if(len(iou_per_gt) != 0):
                max_iou = max(iou_per_gt)
                iou_values.append(max_iou)
                correct_class_cnt += 1 # Do it here so one prediction can only be counted once

        # Calculate mean IoU
        if (len(iou_values) != 0):
            result.iou = sum(iou_values) / len(iou_values)
        else:
            if(len(gt_coords) == 0 and len(predictions) == 0):
                result.iou = 1.0
            else:
                result.iou = 0.0

        # Calculate the accuracy (number of correct predictions vs number of ground truth objects)
        if(len(gt_coords) != 0):
            result.correct_class_ratio = correct_class_cnt/len(gt_coords)
        else:
            result.correct_class_ratio = 0.0
            if(correct_class_cnt == 0):
                result.correct_class_ratio = 1.0

        return result


    def __init__(self):
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='../../YOLOv5s_80batch_640img_noAugment.pt')

        # Define the action space
        action_bounds = (0.3, 1.5)

        self.action_space = Box(low = action_bounds[0],
                               high = action_bounds[1],
                               shape = (1,),
                               dtype = 'float32')

        # Define the observation space for an image
        image_shape = (DistortionEnv.image_width, DistortionEnv.image_height, 3)  # (height, width, channels)
        image_dtype = np.uint8

        # Load training images
        self.train_images = [file for file in os.listdir(train_image_path) if file.endswith(".png")]

        self.observation_space = Box(low = 0, high = 255, shape = image_shape, dtype = np.uint8)

    def reset(self, seed=None):
        # Set amount of possible consecutive actions
        self.remaining_actions = 1

        # Choose random image for this episode
        self.image_name = random.choice(self.train_images)

        # Load input image
        input_image_path = os.path.join(train_image_path, self.image_name)
        self.image_input = self._load_and_convert_image(input_image_path)

        # Load original image
        #orig_image_path = os.path.join(original_image_path, self.image_name)
        #self.image_original = self._load_and_convert_image(orig_image_path)

        # Load ground truth (label and bounding box file)
        label_filename = self.image_name.split('.')[0] + ".txt"
        self.gt_coords = self._get_gt_coords_from_yolo_file(os.path.join(label_path, label_filename))

        self.state = self.image_input.copy()
        return self.state, {}

    def step(self, action):
        self.remaining_actions -= 1

        # change image parameters according to action
        distortion_factor = action
        image = Image.fromarray(self.state)
        image_hsv = image.convert('HSV')
        enhancer = ImageEnhance.Color(image_hsv)
        enhanced_image_hsv = enhancer.enhance(distortion_factor)
        enhanced_image_rgb = enhanced_image_hsv.convert('RGB')
        self.state = np.asarray(enhanced_image_rgb)


        # use detection model on improved image
        results = self.model(self.state)
        predictions = results.pandas().xyxy[0]
        reward_data_improved = self._calculate_iou_and_accuracy(self.gt_coords, predictions)

        # use detection model on input image
        results = self.model(self.image_input)
        predictions = results.pandas().xyxy[0]
        reward_data_input = self._calculate_iou_and_accuracy(self.gt_coords, predictions)

        reward = 0.9 * (reward_data_improved.correct_class_ratio - reward_data_input.correct_class_ratio)
        reward += 0.1 * (reward_data_improved.iou - reward_data_input.iou)
        reward *= 10

        observation = self.state

        if (self.remaining_actions <= 0):
          done = True
        else:
          done = False

        info = {} # Placeholder

        return observation, reward, done, False, info

    def render(self):
        pass

In [8]:
env = DistortionEnv()

Using cache found in C:\Users\phili/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-7 Python-3.10.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)



[Errno 2] No such file or directory: 'd:\\miniconda3\\lib\\site-packages\\pyparsing-3.0.9.dist-info\\METADATA'


Fusing layers... 
Model summary: 213 layers, 7126096 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 


## Load RL Model and predict

In [9]:
# Load the Model
model = SAC.load("../../sac_yolov5_hsv.zip", env = env)
vec_env = model.get_env()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [83]:
input_image_path = "../../Realbilder/21.PNG"
image = env._load_and_convert_image(input_image_path)
Image.fromarray(image).save("../../detection_imgs/distorted_img.png")
#Image.fromarray(image).show()

In [90]:
action, _states = model.predict(image)
print(action)

image_hsv = Image.fromarray(image).convert('HSV')
enhancer = ImageEnhance.Color(image_hsv)
enhanced_image_hsv = enhancer.enhance(action)
enhanced_image_rgb = enhanced_image_hsv.convert('RGB')
enhanced_image_rgb.save("../../detection_imgs/improved_img.png")

[    0.40739]


## Load YOLO Model and predict

In [91]:
!python ../../YOLOv5_GTSDB/yolov5/detect.py --weights ../../YOLOv5s_80batch_640img_noAugment.pt --img 640 --conf 0.2 --source ../../detection_imgs --save-txt --save-conf

detect: weights=['../../YOLOv5s_80batch_640img_noAugment.pt'], source=../../detection_imgs, data=..\..\YOLOv5_GTSDB\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=..\..\YOLOv5_GTSDB\yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-175-g5f11555 Python-3.10.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)

Fusing layers... 
Model summary: 213 layers, 7126096 parameters, 0 gradients, 16.1 GFLOPs
image 1/2 C:\Users\phili\Documents\TU_Wien\Bachelorarbeit_C\detection_imgs\distorted_img.png: 640x640 (no detections), 22.0ms
image 2/2 C:\Users\phili\Documents\TU_Wien\Bachelorarbeit_C\detection_imgs\improved_img.png: 640x640 1 Speed limit (30km/h), 21.0ms
Speed: 1.0ms